# population analysis on ephys units across sessions

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import sys
sys.path.insert(0, '/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys/')

In [3]:
from util.paths import find
from project_analysis.ephys.ephys_utils import load_ephys
from project_analysis.ephys.population_utils import make_unit_summary, make_session_summary

## if the data are spread across directories for individual recordings, and need to be pooled

In [ ]:
# pool data listed in csv file
# df = load_ephys(r'T:/freely_moving_ephys/batch_files/completed_experiment_pool.csv')

In [ ]:
df

In [ ]:
# save as one h5 file
# df.to_hdf(r'T:\freely_moving_ephys\population_summaries\050321/exp_pool_all_data_050321.h5', 'w')

In [ ]:
# make a pdf summary
# make_unit_summary(df, r'T:\freely_moving_ephys\population_summaries\050321')

## if there is already an h5 file of pooled data

In [4]:
# load existing .h5 file
df = pd.read_hdf(r'T:\freely_moving_ephys\population_summaries\060821\reduced_pooled_ephys_data.h5')

In [5]:
# make a pdf summary
make_unit_summary(df, r'T:\freely_moving_ephys\population_summaries\060821')

215it [06:58,  1.95s/it]


saving pdf


In [6]:
make_session_summary(df, r'T:\freely_moving_ephys\population_summaries\060821')

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [05:11<00:00, 23.99s/it]

saving pdf


In [ ]:
df1 = df.copy()

In [ ]:
dEye_series = df1['fm1_dHead']
dHead_series = df1['fm1_dEye']

In [ ]:
df1 = df1.drop(labels=['fm1_dHead','fm1_dEye'], axis=1)

In [ ]:
df1 = df1.assign(fm1_dEye=dEye_series)

In [ ]:
df1 = df1.assign(fm1_dHead=dHead_series)

In [ ]:
cols = df1.columns.values
print(len(cols), len(np.unique(cols)))

In [ ]:
shcols = [c for c in cols if 'gratingssh' in c]
shcols

In [ ]:
for c in shcols:
    new_col = str(c.replace('gratingssh', 'gratings'))
    df1 = df1.rename(columns={str(c): new_col})

In [ ]:
df2 = df1.copy()

In [ ]:
df2 = df2.groupby(lambda x:x, axis=1)

In [ ]:
df2 = df2.agg(np.nansum)

In [ ]:
cols = df2.columns.values

In [ ]:
badcols = []
for c in cols:
    if any(s in c for s in ['fm2','hf5','hf6','hf7','hf8']):
        badcols.append(c)

In [ ]:
badcols

In [ ]:
df2 = df2.drop(labels=badcols, axis=1)

In [ ]:
df2.columns.values

In [ ]:
df2.to_hdf(r'T:\freely_moving_ephys\population_summaries\060821\complete_pooled_ephys_data.h5', 'w')

In [ ]:
np.unique(df2['session'])

In [ ]:
badsessions = ['020121_EE13P1RT_control_Rig2', '021121_EE12P1NN_control_Rig2', '021521_EE11P11LT_control_Rig2',
               '022321_EE12P1LN_control_Rig2','022321_EE13P2RT_control_Rig2','030821_EE8P7LT_control_Rig2',
               '030821_EE8P7RT_control_Rig2','031621_EE8P8RT_control_Rig2','041521_J538LT_control_Rig2']

In [ ]:
df3 = df2[~df2['session'].isin(badsessions)]

In [ ]:
np.unique(df3['session'])

In [ ]:
df3.to_hdf(r'T:\freely_moving_ephys\population_summaries\060821\reduced_pooled_ephys_data.h5', 'w')